# More about vectors and plotting

## Vectors in Julia
You can manipulate vectors (otherwise known as lists) easily in Julia. Adding, subtracting, and dot product are built in. But cross product is not, but this is contained in the LinearAlgebra package.

In [10]:
using LinearAlgebra
using LaTeXStrings, Measures

In [11]:
#default(fontfamily = "Computer Modern", size=(800,500), titlefont = (16), legendfontsize = 10, 
#        guidefont = (16, :darkgreen), tickfont = (12, :black), 
#        framestyle = :box, yminorgrid = true)